# How to Use ArxivReader for Retrieving Papers from arXiv
In this notebook, we will demonstrate how to use the `ArxivReader` class for accessing papers from the arXiv repository. The `ArxivReader` class is designed to interact with the arXiv API and retrieve paper content and metadata, which can be utilized in various research and question-answering systems. We'll be leveraging open-source models available on the internet, such as Mistral, to process the retrieved data.

To begin, ensure you have set up your environment variables and API keys in Python using the dotenv library. This is crucial for securely managing sensitive information, such as API keys, especially when using services like HuggingFace. Ensure your `HUGGINGFACE_API_KEY` is defined in the `.env` file to avoid hardcoding sensitive data into your codebase, thus enhancing security and maintainability.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/osllmai/inDox/blob/master/Demo/arxiv_chroma_semantic.ipynb)

In [ ]:
!pip install indox
!pip install chromadb
!pip install arxiv
!pip install semantic_text_splitter

## Setting Up the Python Environment

If you are running this project in your local IDE, please create a Python environment to ensure all dependencies are correctly managed. You can follow the steps below to set up a virtual environment named `indox`:

### Windows

1. **Create the virtual environment:**
```bash
python -m venv indox
```
2. **Activate the virtual environment:**
```bash
indox_judge\Scripts\activate
```

### macOS/Linux

1. **Create the virtual environment:**
   ```bash
   python3 -m venv indox
    ```

2. **Activate the virtual environment:**
    ```bash
   source indox/bin/activate
    ```
   
### Install Dependencies

Once the virtual environment is activated, install the required dependencies by running:

```bash
pip install -r requirements.txt
```

## Import Essential Libraries

Next, we import the essential libraries for our Indox question-answering system:

- `IndoxRetrievalAugmentation`: Enhances the retrieval process by improving the relevance and quality of the documents retrieved, leading to better QA performance.
- `MistralQA`: A powerful QA model provided by Indox, built on top of the Hugging Face model architecture. It leverages state-of-the-art language understanding to deliver precise answers.
- `HuggingFaceEmbedding`: This library uses Hugging Face embeddings to enrich semantic understanding, making it easier to capture the contextual meaning of the text.
- `SemanticTextSplitter`: utilizes a Hugging Face tokenizer to intelligently split text into chunks based on a specified maximum number of tokens, ensuring that each chunk maintains semantic coherence.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv('api.env')

HUGGINGFACE_API_KEY = os.environ['HUGGINGFACE_API_KEY']

In [2]:
from indox import IndoxRetrievalAugmentation
indox = IndoxRetrievalAugmentation()

INFO: IndoxRetrievalAugmentation initialized

            ██  ███    ██  ██████   ██████  ██       ██
            ██  ████   ██  ██   ██ ██    ██   ██  ██
            ██  ██ ██  ██  ██   ██ ██    ██     ██
            ██  ██  ██ ██  ██   ██ ██    ██   ██   ██
            ██  ██  █████  ██████   ██████  ██       ██
            


## Building the ArxivReader System and Initializing Models

Next, we will build our ArxivReader system and initialize the MistralQA model along with the HuggingFaceEmbedding model. This setup will enable us to effectively retrieve and process arXiv papers, leveraging the advanced capabilities of these models for our question-answering tasks.


In [3]:
from indox.llms import HuggingFaceModel
from indox.embeddings import HuggingFaceEmbedding
mistral_qa = HuggingFaceModel(api_key=HUGGINGFACE_API_KEY,model="mistralai/Mistral-7B-Instruct-v0.2")
embed = HuggingFaceEmbedding(api_key=HUGGINGFACE_API_KEY,model="multi-qa-mpnet-base-cos-v1")

INFO: Initializing HuggingFaceModel with model: mistralai/Mistral-7B-Instruct-v0.2
INFO: HuggingFaceModel initialized successfully
INFO: Initialized HuggingFaceEmbedding with model: multi-qa-mpnet-base-cos-v1


## Setting Up the ArxivReader for Retrieving Papers
To demonstrate the capabilities of our `ArxivReader` system and `Indox` question-answering model, we will use a sample paper ID. This paper will contain arXiv paper, which we will use for testing and evaluation.

In [4]:
from indox.data_connectors import ArxivReader

reader = ArxivReader()

paper_ids = ["2201.08239"]
documents = reader.load_content(paper_ids)

In [5]:
content = documents

## Splitting Content into Manageable Chunks
We use the `SemanticTextSplitter` function from the `indox.splitter` module to divide the retrieved content into smaller, meaningful chunks.

In [6]:
from indox.splitter import SemanticTextSplitter
splitter = SemanticTextSplitter(400)
content_chunks = splitter.split_text(content)

## Storing and Indexing Content with Chroma
We use the `Chroma` vector store from the `indox.vector_stores` module to store and index the content chunks. By creating a collection named "sample" and applying an embedding function (`embed`), we convert each chunk into a vector for efficient retrieval. The `add` method then adds these vectors to the database, enabling scalable and effective search for question-answering tasks.

In [7]:
from indox.vector_stores import Chroma
db = Chroma(collection_name="sample",embedding_function=embed)
db.add(docs=content_chunks)

INFO: Storing documents in the vector store
INFO: Embedding documents
INFO: Starting to fetch embeddings for texts using model: SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
INFO: Document added successfully to the vector store.
INFO: Documents stored successfully


## Querying the Arxiv Data with Indox
With our `ArxivReader` system and `Indox` setup complete, we are ready to test it using a sample query. This test will show how well our system can retrieve and generate accurate answers based on the arXiv papers stored in the vector store.

We’ll use a sample query to evaluate our system:
- **Query**: "what are challenges?"

This question will be processed by the `ArxivReader` and `Indox` system to retrieve relevant papers and generate a precise response based on the information.

Let’s test our setup with this query.

In [8]:
query = "what are challenges?"
retriever = indox.QuestionAnswer(vector_database=db, llm=mistral_qa, top_k=2)

Now that our `ArxivReader` system with `Indox` is fully set up, we can test it with a sample query. We’ll use the `invoke` method to get a response from the system.
The `invoke` method processes the query using the connected QA model and retrieves relevant information from the vector store.

We’ll pass the query to the invoke method and print the response to evaluate how effectively the system retrieves and generates answers.

In [9]:
answer = retriever.invoke(query)
context = retriever.context

INFO: Retrieving context and scores from the vector database
INFO: Embedding documents
INFO: Starting to fetch embeddings for texts using model: SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
INFO: Generating answer without document relevancy filter
INFO: Answering question
INFO: Sending request to Hugging Face API
INFO: Received successful response from Hugging Face API
INFO: Query answered successfully


In [10]:
answer

"The challenges addressed in the article are safety and factual grounding for LaMDA, a family of Transformer-based neural language models specialized for dialog applications. Safety challenges include ensuring that the model's responses are consistent with a set of human values, such as preventing harmful suggestions and unfair bias. Factual grounding challenges involve enabling the model to consult external knowledge sources, such as an information retrieval system, a language translator, and a calculator."